In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
from google.colab import files

In [ ]:
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

Saving df_combined.csv to df_combined.csv


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/df_combined.csv')

In [ ]:
df.head()

,lat,lon,year,doy,uv_index,temp_2m,dew_2m,precipitation_mm,humidity_2m,surface_pressure,wind_speed_2m,surface_soil_wetness,root_soil_wetness
0,14.25,72.75,2015,274,2.02,28.00,25.18,13.40,19.96,100.93,3.25,1.00,-999.00
1,14.25,73.25,2015,274,2.16,27.78,25.02,13.53,19.78,100.94,3.16,1.00,-999.00
2,14.25,73.75,2015,274,2.16,27.58,24.84,15.78,19.53,100.91,3.11,0.99,-999.00
3,14.25,74.25,2015,274,1.88,26.74,24.20,17.68,19.04,99.72,2.23,0.95,0.94
4,14.25,74.75,2015,274,1.88,25.31,23.12,14.13,18.35,97.04,1.40,0.91,0.93


In [ ]:
class WeatherDataset(Dataset):
    def __init__(self, csv_file):
        data = pd.read_csv(csv_file)
        self.features = data[['lat', 'lon', 'year', 'doy', 'uv_index', 'temp_2m',
                              'dew_2m', 'humidity_2m', 'surface_pressure',
                              'wind_speed_2m', 'surface_soil_wetness',
                              'root_soil_wetness']].values
        self.targets = data[['precipitation_mm', 'temp_2m', 'dew_2m',
                             'humidity_2m', 'surface_pressure',
                             'wind_speed_2m', 'surface_soil_wetness',
                             'root_soil_wetness']].values

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        target = self.targets[idx]
        return torch.tensor(feature, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)

In [ ]:
class AFNO1DNet(nn.Module):
    def __init__(self, in_chans=12, out_chans=8, hidden_size=64):
        super(AFNO1DNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=in_chans, out_channels=hidden_size, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=hidden_size, out_channels=hidden_size, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=hidden_size, out_channels=out_chans, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.bn2 = nn.BatchNorm1d(out_chans)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn2(x)
        return x

In [ ]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, targets in dataloader:
            inputs = inputs.unsqueeze(1)  # Add the channel dimension
            targets = targets.unsqueeze(1)  # Add the channel dimension to targets

            optimizer.zero_grad()
            outputs = model(inputs.permute(0, 2, 1))  # Rearranging to (batch_size, channels, sequence_length)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader)}")

In [ ]:
def get_user_input():
    lat = float(input("Enter Latitude: "))
    lon = float(input("Enter Longitude: "))
    year = int(input("Enter Year: "))
    doy = int(input("Enter Day of Year (1-365): "))
    return lat, lon, year, doy

In [ ]:
THRESHOLDS = {
    'uv_index': 8,
    'temp_2m': 35,
    'dew_2m': 2,
    'precipitation_mm': 50,
    'humidity_2m': 90,
    'surface_pressure': 1000,
    'wind_speed_2m': 20,
    'surface_soil_wetness': 0.9,
    'root_soil_wetness': 0.9
}

ALERT_MESSAGES = {
    'uv_index': "High UV Index Alert! Please take necessary precautions.",
    'temp_2m': "Heatwave Alert! Temperature is predicted to rise above 35°C.",
    'dew_2m': "Frost Alert! Dew point is very low.",
    'precipitation_mm': "Heavy Rainfall Alert! Precipitation is predicted to exceed 50mm.",
    'humidity_2m': "High Humidity Alert! Humidity levels are above 90%.",
    'surface_pressure': "Low Pressure Alert! There may be a storm or extreme weather.",
    'wind_speed_2m': "Strong Wind Alert! Wind speeds exceed 20 m/s.",
    'surface_soil_wetness': "Flood Risk Alert! Surface soil wetness is very high.",
    'root_soil_wetness': "Waterlogging Alert! Root soil wetness is very high."
}

In [ ]:
def generate_alerts(predictions):
    alerts = []
    for i, (var, threshold) in enumerate(THRESHOLDS.items()):
        if i < len(predictions):  # Ensure no out-of-bounds indexing
            predicted_value = predictions[i]
            if predicted_value > threshold:
                alerts.append(ALERT_MESSAGES[var])
    return alerts

In [ ]:
def make_prediction(model, dataloader):
    model.eval()
    all_predictions = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.unsqueeze(2)  # Add the necessary dimension
            outputs = model(inputs)
            predictions = outputs.squeeze(2).numpy()
            all_predictions.append(predictions)
    return np.concatenate(all_predictions, axis=0)


In [ ]:
def predict(model, lat, lon, year, doy):
    model.eval()
    # Construct input tensor
    input_tensor = torch.tensor([[lat, lon, year, doy, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.float32)
    input_tensor = input_tensor.unsqueeze(1)  # Add the channel dimension

    with torch.no_grad():
        output = model(input_tensor.permute(0, 2, 1))  # Rearranging for Conv1d

    return output.squeeze().numpy()

# Define the AFNO1DNet model (you already have this defined in your code)
model = AFNO1DNet(in_chans=12, out_chans=8)  # Load your trained model

# Load trained model weights if you have a saved model (optional)
# model.load_state_dict(torch.load('afno1dnet_model.pth'))  # Uncomment if you have saved weights

# Take user input
lat, lon, year, doy = get_user_input()

# Make the prediction
predictions = predict(model, lat, lon, year, doy)

# Print the predictions
print("Predicted values for the next day:")
for i, (var, value) in enumerate(zip(THRESHOLDS.keys(), predictions)):
    print(f"{var}: {value}")

# Generate alerts based on predictions
alerts = generate_alerts(predictions)

# Display the alerts if any
if alerts:
    print("\nWeather Alerts:")
    for alert in alerts:
        print(alert)
else:
    print("No weather alerts for the given location.")

Enter Latitude: 25.45
Enter Longitude: 75.34
Enter Year: 2021
Enter Day of Year (1-365): 275
Predicted values for the next day:
uv_index: -0.7210561037063599
temp_2m: -7.854537010192871
dew_2m: 1.139124870300293
precipitation_mm: 3.1505839824676514
humidity_2m: 14.863171577453613
surface_pressure: -3.2177822589874268
wind_speed_2m: 9.51456356048584
surface_soil_wetness: -8.438031196594238
No weather alerts for the given location.


In [28]:
pip install streamlit


In [29]:
!wget -q -O - ipv4.icanhazip.com

34.16.251.195


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.251.195:8501

Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) y
your url is: https://clever-radios-relate.loca.lt
